# Data Prepration

In [26]:
from config import Config
import utils
from utils import EnvHndler
import os
import sympy as sp
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from datahandler import MathData, create_data_loader
import random

In [27]:
def set_seed(seed):
    """Set seed"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)
set_seed(0)

In [33]:
config = Config()

In [39]:
config.device.type == 'cpu'

True

In [29]:
config.reload_size = 10

In [30]:
env = EnvHndler(config)

In [133]:
df = MathData(config, env, dtype = 'train')

In [134]:
df.data

[["sub Y' INT+ 1", 'x'],
 ["sub Y' INT+ 2", 'mul INT+ 2 x'],
 ["sub Y' mul INT+ 4 x", 'mul INT+ 2 pow x INT+ 2'],
 ["sub Y' add INT+ 2 mul INT+ 2 x", 'add pow x INT+ 2 mul INT+ 2 x'],
 ["sub Y' mul INT+ 3 pow x INT+ 2", 'pow x INT+ 3'],
 ["sub Y' pow x INT- 1", 'ln x'],
 ["sub Y' mul INT- 1 sin x", 'cos x'],
 ["sub Y' exp x", 'exp x'],
 ["sub Y' cos x", 'sin x'],
 ["sub Y' add INT+ 1 mul INT+ 4 x", 'add x mul INT+ 2 pow x INT+ 2']]

In [135]:
[env.word2id[x] for x in df.data[1][0].split()]

[67, 79, 71, 83]

In [136]:
len(df)

1152921504606846976

In [137]:
df[1000]

(['sub', "Y'", 'pow', 'x', 'INT-', '1'], ['ln', 'x'])

In [32]:
loader = create_data_loader(config, env, dtype = 'valid')

In [139]:
it = iter(loader)

In [140]:
config.batch_size

10

In [141]:
batch = next(it)

In [142]:
def get_masks(slen, lengths, causal):
    """
    Generate hidden states mask, and optionally an attention mask.
    """
    assert lengths.max().item() <= slen
    bs = lengths.size(0)
    alen = torch.arange(slen, dtype=torch.long, device=lengths.device)
    mask = alen < lengths[:, None]

    # attention mask is the same as mask, or triangular inferior attention (causal)
    if causal:
        attn_mask = alen[None, None, :].repeat(bs, slen, 1) <= alen[None, :, None]
    else:
        attn_mask = mask

    # sanity check
    assert mask.size() == (bs, slen)
    assert causal is False or attn_mask.size() == (bs, slen, slen)

    return mask, attn_mask

In [143]:
(x,len_x), (y,len_y), _ = batch

In [144]:
slen = x.shape[0]

In [172]:
mask, am = get_masks(slen, len_x, causal = True)

In [173]:
len_x[6]

tensor(8)

In [174]:
 alen = torch.arange(slen, dtype=torch.long)

In [179]:
am[0]

tensor([[ True, False, False, False, False, False, False, False, False, False,
         False],
        [ True,  True, False, False, False, False, False, False, False, False,
         False],
        [ True,  True,  True, False, False, False, False, False, False, False,
         False],
        [ True,  True,  True,  True, False, False, False, False, False, False,
         False],
        [ True,  True,  True,  True,  True, False, False, False, False, False,
         False],
        [ True,  True,  True,  True,  True,  True, False, False, False, False,
         False],
        [ True,  True,  True,  True,  True,  True,  True, False, False, False,
         False],
        [ True,  True,  True,  True,  True,  True,  True,  True, False, False,
         False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True, False,
         False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         False],
        [ True,  True,  True,  True,  Tr

In [168]:
mask

tensor([[ True,  True,  True,  True,  True,  True, False, False, False, False,
         False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True],
        [ True,  True,  True,  True,  True,  True, False, False, False, False,
         False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True],
        [ True,  True,  True,  True,  True,  True,  True,  True, False, False,
         False],
        [ True,  True,  True,  True,  True,  True,  True,  True, False, False,
         False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True],
        [ True,  True,  True,  True,  True,  True, False, False, False, False,
         False]])

In [166]:
alen < len_x[:]

RuntimeError: The size of tensor a (11) must match the size of tensor b (10) at non-singleton dimension 0

# Model Prepration

In [1]:
from config import Config
import torch
import torch.nn as nn
import os
import pickle
import numpy as np
import random
import torch.nn.functional as F
import math
from Model import Multi_Head_Attention, Feed_Forward, get_masks, EncoderBlock, DecoderBlock, Transformers
from utils import EnvHndler
from trainer import Trainer
from evaluator import Evaluator
import json

In [2]:
config = Config()

In [3]:
logger = config.get_logger()

INFO - 04/28/21 09:14:29 - 0:00:00 - ============ Initialized logger ============
INFO - 04/28/21 09:14:29 - 0:00:00 - Dataset_dir: /home/mehrazin/Documents/Deep_Learning_Project/Repository/Symbolic-Mathematics/Dataset/Prime_BWD
                                     batch_size: 10
                                     beam_eval: False
                                     clean_prefix_expr: True
                                     clip_grad_norm: 5
                                     command: python /home/mehrazin/anaconda3/envs/DAV/lib/python3.8/site-packages/ipykernel_launcher.py '-f' '/home/mehrazin/.local/share/jupyter/runtime/kernel-969b6830-ddcf-4369-a116-43e9156fe544.json' --exp_id "8"
                                     device: cpu
                                     dump_path: /home/mehrazin/Documents/Deep_Learning_Project/Repository/Symbolic-Mathematics/Dumped
                                     env_seed: 0
                                     eos_index: 0
                 

In [4]:
env = EnvHndler(config)

In [5]:
config.n_words

91

In [6]:
bs = 4
# qlen = 5
ilen = 5
olen = 3
config.model_dim = 16
dm = config.model_dim
head_dim = dm//config.num_head
x = torch.randint(low = 0, high = config.n_words, size = (ilen, bs)).long()
y = torch.randint(low = 0, high = config.n_words, size = (olen, bs)).long()
len_x = torch.tensor([2,4,5,3])
len_y = torch.tensor([1,3,2,3])
# Q = torch.randn((bs, qlen, dm))
# V = torch.randn((bs, qlen, dm))
# K = torch.randn((bs, qlen, dm))
# lenghts = torch.tensor([2,4,5,3])

In [7]:
model = Transformers(config)

In [8]:
config.max_epoch = 100
config.reload_size = 32
config.batch_size = 32
config.epoch_size = 32

In [9]:
trainer = Trainer(config, env, model)

In [10]:
trainer.metrics

[('valid_prim_bwd_acc', True)]

In [11]:
evaluator = Evaluator(trainer)

# evaluation
if config.eval_only:
    scores = evaluator.run_all_evals()
    for k, v in scores.items():
        logger.info("%s -> %.6f" % (k, v))
    logger.info("__log__:%s" % json.dumps(scores))
    exit()

# training
for _ in range(config.max_epoch):

    logger.info("============ Starting epoch %i ... ============" % trainer.epoch)

    trainer.n_equations = 0

    while trainer.n_equations < trainer.epoch_size:
        # training steps
        trainer.enc_dec_step()
        trainer.iter()

    logger.info("============ End of epoch %i ============" % trainer.epoch)

    # evaluate perplexity
    scores = evaluator.run_all_evals()

#     print / JSON log
    for k, v in scores.items():
        logger.info("%s -> %.6f" % (k, v))

    logger.info("__log__:%s" % json.dumps(scores))

#     end of epoch
    trainer.save_best_model(scores)
    trainer.save_periodic()
    trainer.end_epoch(scores)

INFO - 04/28/21 09:14:37 - 0:00:08 - ============ Starting epoch 0 ... ============
INFO - 04/28/21 09:14:37 - 0:00:08 - ============ End of epoch 0 ============
INFO - 04/28/21 09:14:37 - 0:00:08 - 0/500
INFO - 04/28/21 09:14:37 - 0:00:08 - 128/500
INFO - 04/28/21 09:14:37 - 0:00:09 - 224/500
INFO - 04/28/21 09:14:37 - 0:00:09 - 320/500
INFO - 04/28/21 09:14:37 - 0:00:09 - 416/500
INFO - 04/28/21 09:14:38 - 0:00:09 - 0/500 (0.0%) equations were evaluated correctly.
INFO - 04/28/21 09:14:38 - 0:00:09 - 0/500
INFO - 04/28/21 09:14:38 - 0:00:09 - 128/500
INFO - 04/28/21 09:14:38 - 0:00:09 - 224/500
INFO - 04/28/21 09:14:38 - 0:00:09 - 320/500
INFO - 04/28/21 09:14:38 - 0:00:10 - 416/500
INFO - 04/28/21 09:14:38 - 0:00:10 - 0/500 (0.0%) equations were evaluated correctly.
INFO - 04/28/21 09:14:38 - 0:00:10 - epoch -> 0.000000
INFO - 04/28/21 09:14:38 - 0:00:10 - valid_prim_bwd_xe_loss -> 60.181211
INFO - 04/28/21 09:14:38 - 0:00:10 - valid_prim_bwd_acc -> 0.000000
INFO - 04/28/21 09:14:38

INFO - 04/28/21 09:14:39 - 0:00:10 - 128/500
INFO - 04/28/21 09:14:39 - 0:00:10 - 224/500
INFO - 04/28/21 09:14:39 - 0:00:10 - 320/500
INFO - 04/28/21 09:14:39 - 0:00:10 - 416/500
INFO - 04/28/21 09:14:39 - 0:00:11 - 0/500 (0.0%) equations were evaluated correctly.
INFO - 04/28/21 09:14:39 - 0:00:11 - 0/500
INFO - 04/28/21 09:14:39 - 0:00:11 - 128/500
INFO - 04/28/21 09:14:39 - 0:00:11 - 224/500
INFO - 04/28/21 09:14:39 - 0:00:11 - 320/500
INFO - 04/28/21 09:14:40 - 0:00:11 - 416/500
INFO - 04/28/21 09:14:40 - 0:00:11 - 0/500 (0.0%) equations were evaluated correctly.
INFO - 04/28/21 09:14:40 - 0:00:11 - epoch -> 1.000000
INFO - 04/28/21 09:14:40 - 0:00:11 - valid_prim_bwd_xe_loss -> 59.586191
INFO - 04/28/21 09:14:40 - 0:00:11 - valid_prim_bwd_acc -> 0.000000
INFO - 04/28/21 09:14:40 - 0:00:11 - valid_prim_bwd_acc_2 -> 0.000000
INFO - 04/28/21 09:14:40 - 0:00:11 - valid_prim_bwd_acc_3 -> 0.000000
INFO - 04/28/21 09:14:40 - 0:00:11 - valid_prim_bwd_acc_4 -> 0.000000
INFO - 04/28/21 09:

INFO - 04/28/21 09:14:41 - 0:00:13 - valid_prim_bwd_acc_4 -> 0.000000
INFO - 04/28/21 09:14:41 - 0:00:13 - valid_prim_bwd_acc_5 -> 0.000000
INFO - 04/28/21 09:14:41 - 0:00:13 - valid_prim_bwd_acc_6 -> 0.000000
INFO - 04/28/21 09:14:41 - 0:00:13 - valid_prim_bwd_acc_7 -> 0.000000
INFO - 04/28/21 09:14:41 - 0:00:13 - valid_prim_bwd_acc_8 -> 0.000000
INFO - 04/28/21 09:14:41 - 0:00:13 - valid_prim_bwd_acc_9 -> 0.000000
INFO - 04/28/21 09:14:41 - 0:00:13 - valid_prim_bwd_acc_10 -> 0.000000
INFO - 04/28/21 09:14:41 - 0:00:13 - valid_prim_bwd_acc_11 -> 0.000000
INFO - 04/28/21 09:14:41 - 0:00:13 - valid_prim_bwd_acc_12 -> 0.000000
INFO - 04/28/21 09:14:41 - 0:00:13 - valid_prim_bwd_acc_13 -> 0.000000
INFO - 04/28/21 09:14:41 - 0:00:13 - valid_prim_bwd_acc_14 -> 0.000000
INFO - 04/28/21 09:14:41 - 0:00:13 - valid_prim_bwd_acc_15 -> 0.000000
INFO - 04/28/21 09:14:41 - 0:00:13 - valid_prim_bwd_acc_16 -> 0.000000
INFO - 04/28/21 09:14:41 - 0:00:13 - valid_prim_bwd_acc_17 -> 0.000000
INFO - 04/28

INFO - 04/28/21 09:14:43 - 0:00:14 - valid_prim_bwd_acc_17 -> 0.000000
INFO - 04/28/21 09:14:43 - 0:00:14 - valid_prim_bwd_acc_18 -> 0.000000
INFO - 04/28/21 09:14:43 - 0:00:14 - valid_prim_bwd_acc_19 -> 0.000000
INFO - 04/28/21 09:14:43 - 0:00:14 - valid_prim_bwd_acc_20 -> 0.000000
INFO - 04/28/21 09:14:43 - 0:00:14 - valid_prim_bwd_acc_21 -> 0.000000
INFO - 04/28/21 09:14:43 - 0:00:14 - valid_prim_bwd_acc_22 -> 0.000000
INFO - 04/28/21 09:14:43 - 0:00:14 - valid_prim_bwd_acc_23 -> 0.000000
INFO - 04/28/21 09:14:43 - 0:00:14 - valid_prim_bwd_acc_24 -> 0.000000
INFO - 04/28/21 09:14:43 - 0:00:14 - valid_prim_bwd_acc_25 -> 0.000000
INFO - 04/28/21 09:14:43 - 0:00:14 - valid_prim_bwd_acc_26 -> 0.000000
INFO - 04/28/21 09:14:43 - 0:00:14 - valid_prim_bwd_acc_27 -> 0.000000
INFO - 04/28/21 09:14:43 - 0:00:14 - valid_prim_bwd_acc_28 -> 0.000000
INFO - 04/28/21 09:14:43 - 0:00:14 - valid_prim_bwd_acc_30 -> 0.000000
INFO - 04/28/21 09:14:43 - 0:00:14 - valid_prim_bwd_acc_31 -> 0.000000
INFO -

INFO - 04/28/21 09:14:44 - 0:00:16 - valid_prim_bwd_acc_31 -> 0.000000
INFO - 04/28/21 09:14:44 - 0:00:16 - test_prim_bwd_xe_loss -> 58.780832
INFO - 04/28/21 09:14:44 - 0:00:16 - test_prim_bwd_acc -> 0.000000
INFO - 04/28/21 09:14:44 - 0:00:16 - test_prim_bwd_acc_3 -> 0.000000
INFO - 04/28/21 09:14:44 - 0:00:16 - test_prim_bwd_acc_4 -> 0.000000
INFO - 04/28/21 09:14:44 - 0:00:16 - test_prim_bwd_acc_5 -> 0.000000
INFO - 04/28/21 09:14:44 - 0:00:16 - test_prim_bwd_acc_6 -> 0.000000
INFO - 04/28/21 09:14:44 - 0:00:16 - test_prim_bwd_acc_7 -> 0.000000
INFO - 04/28/21 09:14:44 - 0:00:16 - test_prim_bwd_acc_8 -> 0.000000
INFO - 04/28/21 09:14:44 - 0:00:16 - test_prim_bwd_acc_9 -> 0.000000
INFO - 04/28/21 09:14:44 - 0:00:16 - test_prim_bwd_acc_10 -> 0.000000
INFO - 04/28/21 09:14:44 - 0:00:16 - test_prim_bwd_acc_11 -> 0.000000
INFO - 04/28/21 09:14:44 - 0:00:16 - test_prim_bwd_acc_12 -> 0.000000
INFO - 04/28/21 09:14:44 - 0:00:16 - test_prim_bwd_acc_13 -> 0.000000
INFO - 04/28/21 09:14:44 - 

...

INFO - 04/28/21 09:16:51 - 0:02:22 - valid_prim_bwd_acc_10 -> 0.000000
INFO - 04/28/21 09:16:51 - 0:02:22 - valid_prim_bwd_acc_11 -> 0.000000
INFO - 04/28/21 09:16:51 - 0:02:22 - valid_prim_bwd_acc_12 -> 0.000000
INFO - 04/28/21 09:16:51 - 0:02:22 - valid_prim_bwd_acc_13 -> 0.000000
INFO - 04/28/21 09:16:51 - 0:02:22 - valid_prim_bwd_acc_14 -> 0.000000
INFO - 04/28/21 09:16:51 - 0:02:22 - valid_prim_bwd_acc_15 -> 0.000000
INFO - 04/28/21 09:16:51 - 0:02:22 - valid_prim_bwd_acc_16 -> 0.000000
INFO - 04/28/21 09:16:51 - 0:02:22 - valid_prim_bwd_acc_17 -> 0.000000
INFO - 04/28/21 09:16:51 - 0:02:22 - valid_prim_bwd_acc_18 -> 0.000000
INFO - 04/28/21 09:16:51 - 0:02:22 - valid_prim_bwd_acc_19 -> 0.000000
INFO - 04/28/21 09:16:51 - 0:02:22 - valid_prim_bwd_acc_20 -> 0.000000
INFO - 04/28/21 09:16:51 - 0:02:22 - valid_prim_bwd_acc_21 -> 0.000000
INFO - 04/28/21 09:16:51 - 0:02:22 - valid_prim_bwd_acc_22 -> 0.000000
INFO - 04/28/21 09:16:51 - 0:02:22 - valid_prim_bwd_acc_23 -> 0.000000
INFO -

INFO - 04/28/21 09:16:53 - 0:02:24 - valid_prim_bwd_acc_23 -> 0.000000
INFO - 04/28/21 09:16:53 - 0:02:24 - valid_prim_bwd_acc_24 -> 0.000000
INFO - 04/28/21 09:16:53 - 0:02:24 - valid_prim_bwd_acc_25 -> 0.000000
INFO - 04/28/21 09:16:53 - 0:02:24 - valid_prim_bwd_acc_26 -> 0.000000
INFO - 04/28/21 09:16:53 - 0:02:24 - valid_prim_bwd_acc_27 -> 0.000000
INFO - 04/28/21 09:16:53 - 0:02:24 - valid_prim_bwd_acc_28 -> 0.000000
INFO - 04/28/21 09:16:53 - 0:02:24 - valid_prim_bwd_acc_30 -> 0.000000
INFO - 04/28/21 09:16:53 - 0:02:24 - valid_prim_bwd_acc_31 -> 0.000000
INFO - 04/28/21 09:16:53 - 0:02:24 - test_prim_bwd_xe_loss -> 47.108547
INFO - 04/28/21 09:16:53 - 0:02:24 - test_prim_bwd_acc -> 0.000000
INFO - 04/28/21 09:16:53 - 0:02:24 - test_prim_bwd_acc_3 -> 0.000000
INFO - 04/28/21 09:16:53 - 0:02:24 - test_prim_bwd_acc_4 -> 0.000000
INFO - 04/28/21 09:16:53 - 0:02:24 - test_prim_bwd_acc_5 -> 0.000000
INFO - 04/28/21 09:16:53 - 0:02:24 - test_prim_bwd_acc_6 -> 0.000000
INFO - 04/28/21 0

INFO - 04/28/21 09:16:54 - 0:02:25 - test_prim_bwd_acc_6 -> 0.000000
INFO - 04/28/21 09:16:54 - 0:02:25 - test_prim_bwd_acc_7 -> 0.000000
INFO - 04/28/21 09:16:54 - 0:02:25 - test_prim_bwd_acc_8 -> 0.000000
INFO - 04/28/21 09:16:54 - 0:02:25 - test_prim_bwd_acc_9 -> 0.000000
INFO - 04/28/21 09:16:54 - 0:02:25 - test_prim_bwd_acc_10 -> 0.000000
INFO - 04/28/21 09:16:54 - 0:02:25 - test_prim_bwd_acc_11 -> 0.000000
INFO - 04/28/21 09:16:54 - 0:02:25 - test_prim_bwd_acc_12 -> 0.000000
INFO - 04/28/21 09:16:54 - 0:02:25 - test_prim_bwd_acc_13 -> 0.000000
INFO - 04/28/21 09:16:54 - 0:02:25 - test_prim_bwd_acc_14 -> 0.000000
INFO - 04/28/21 09:16:54 - 0:02:25 - test_prim_bwd_acc_15 -> 0.000000
INFO - 04/28/21 09:16:54 - 0:02:25 - test_prim_bwd_acc_16 -> 0.000000
INFO - 04/28/21 09:16:54 - 0:02:25 - test_prim_bwd_acc_17 -> 0.000000
INFO - 04/28/21 09:16:54 - 0:02:25 - test_prim_bwd_acc_18 -> 0.000000
INFO - 04/28/21 09:16:54 - 0:02:25 - test_prim_bwd_acc_19 -> 0.000000
INFO - 04/28/21 09:16:54

INFO - 04/28/21 09:16:55 - 0:02:27 - test_prim_bwd_acc_19 -> 0.000000
INFO - 04/28/21 09:16:55 - 0:02:27 - test_prim_bwd_acc_20 -> 0.000000
INFO - 04/28/21 09:16:55 - 0:02:27 - test_prim_bwd_acc_21 -> 0.000000
INFO - 04/28/21 09:16:55 - 0:02:27 - test_prim_bwd_acc_22 -> 0.000000
INFO - 04/28/21 09:16:55 - 0:02:27 - test_prim_bwd_acc_23 -> 0.000000
INFO - 04/28/21 09:16:55 - 0:02:27 - test_prim_bwd_acc_24 -> 0.000000
INFO - 04/28/21 09:16:55 - 0:02:27 - test_prim_bwd_acc_25 -> 0.000000
INFO - 04/28/21 09:16:55 - 0:02:27 - test_prim_bwd_acc_26 -> 0.000000
INFO - 04/28/21 09:16:55 - 0:02:27 - test_prim_bwd_acc_27 -> 0.000000
INFO - 04/28/21 09:16:55 - 0:02:27 - test_prim_bwd_acc_28 -> 0.000000
INFO - 04/28/21 09:16:55 - 0:02:27 - test_prim_bwd_acc_31 -> 0.000000
INFO - 04/28/21 09:16:55 - 0:02:27 - test_prim_bwd_acc_36 -> 0.000000
INFO - 04/28/21 09:16:55 - 0:02:27 - __log__:{"epoch": 97, "valid_prim_bwd_xe_loss": 46.517709750652315, "valid_prim_bwd_acc": 0.0, "valid_prim_bwd_acc_2": 0.0,

INFO - 04/28/21 09:16:57 - 0:02:29 - Saving checkpoint to /home/mehrazin/Documents/Deep_Learning_Project/Repository/Symbolic-Mathematics/Dumped/8/checkpoint.pth ...
WARNING - 04/28/21 09:16:57 - 0:02:29 - Saving Model parameters ...
WARNING - 04/28/21 09:16:57 - 0:02:29 - Saving Model optimizer ...
INFO - 04/28/21 09:16:57 - 0:02:29 - ============ Starting epoch 99 ... ============
INFO - 04/28/21 09:16:57 - 0:02:29 - ============ End of epoch 99 ============
INFO - 04/28/21 09:16:57 - 0:02:29 - 0/500
INFO - 04/28/21 09:16:57 - 0:02:29 - 128/500
INFO - 04/28/21 09:16:58 - 0:02:29 - 224/500
INFO - 04/28/21 09:16:58 - 0:02:29 - 320/500
INFO - 04/28/21 09:16:58 - 0:02:29 - 416/500
INFO - 04/28/21 09:16:58 - 0:02:29 - 0/500 (0.0%) equations were evaluated correctly.
INFO - 04/28/21 09:16:58 - 0:02:29 - 0/500
INFO - 04/28/21 09:16:58 - 0:02:30 - 128/500
INFO - 04/28/21 09:16:58 - 0:02:30 - 224/500
INFO - 04/28/21 09:16:58 - 0:02:30 - 320/500
INFO - 04/28/21 09:16:58 - 0:02:30 - 416/500
INFO

In [12]:
trainer.stats

OrderedDict([('processed_e', 3200),
             ('processed_w', 48245),
             ('prim_bwd',
              [4.930577754974365,
               4.792444229125977,
               4.777329921722412,
               4.7190046310424805,
               4.687132358551025,
               4.666828632354736,
               4.618255138397217,
               4.576709270477295,
               4.4589691162109375,
               4.465731143951416,
               4.462161064147949,
               4.335447788238525,
               4.302576541900635,
               4.3240485191345215,
               4.323019027709961,
               4.313997745513916,
               4.271739482879639,
               4.246503829956055,
               4.219818115234375,
               4.136155128479004,
               4.162965774536133,
               4.176421642303467,
               4.174023628234863,
               4.167017459869385,
               4.114540100097656,
               4.033487319946289,
              

In [7]:
enc = tr(mode = 'encode', x = x, len_x = len_x)

In [9]:
dec = tr(mode = 'decode', y = y, len_y = len_y, encoded = enc.transpose(0,1), len_enc = len_x)

In [11]:
# target words to predict
alen = torch.arange(len_y.max(), dtype=torch.long, device=len_y.device)
pred_mask = alen[:, None] < len_y[None] - 1  # do not predict anything given the last target word
t = y[1:].masked_select(pred_mask[:-1])
assert len(t) == (len_y - 1).sum().item()

In [23]:
scores, loss = tr(mode = 'predict', tensor = dec, pred_mask = pred_mask, y = t, get_scores = True)

In [15]:
t = input("Continue training? [y/n]")

Continue training? [y/n]y


In [16]:
t

'y'